## Parameter recovery

We simulated data for a set of design and various ground truth parameters. Now we will try to estimate those parameters from the simulated data

In [40]:
# Built-in/Generic Imports
import os,sys
import glob

# Libs
import numpy as np
import pandas as pd
import pymc as pm
import arviz as az

In [41]:

def estimate_bhm(index,design_df,choices):

    # save_dir = os.path.join('simul','bhm')
    
    delay_amt = design_df['cdd_delay_amt'].values
    delay_wait = design_df['cdd_delay_wait'].values
    immed_amt = design_df['cdd_immed_amt'].values
    immed_wait = design_df['cdd_immed_wait'].values
    subj_id = [index]*len(choices)
    # We will fit a model for each subject
    with pm.Model() as model_simple:

        # Hyperparameters for kappa
        mu_kappa_hyper = pm.Beta('mu_kappa_hyper',mu=0.02,sigma=0.05)
        # use the same hyper SD for both parameters
        sd_hyper = pm.LogNormal('sd_hyper',sigma=1)

        kappa = pm.LogNormal('kappa',mu=mu_kappa_hyper,sigma=sd_hyper,shape=np.size(np.unique(subj_id)))
        gamma = pm.HalfNormal('gamma',sigma=sd_hyper,shape=np.size(np.unique(subj_id)))
        
        prob = pm.Deterministic('prob', 1 / (1 + pm.math.exp(-gamma[subj_id] * ( delay_amt/(1+(kappa[subj_id]*delay_wait)) 
                                                                                - immed_amt/(1+(kappa[subj_id]*immed_wait)) ))))

        y_1 = pm.Bernoulli('y_1',p=prob,observed=choices)

        trace_prior = pm.sample(1000, tune=100, cores=2,target_accept=0.98,progressbar=False)


    # This is how you get a nice array. Note that this returns a pandas DataFrame, not a numpy array. Indexing is totally different.
    summary= az.summary(trace_prior,round_to=10)
    kappa_hat = summary['mean'].loc['kappa[{}]'.format(index)]
    gamma_hat = summary['mean'].loc['gamma[{}]'.format(index)]
    return kappa_hat,gamma_hat
    # fn = os.path.join(save_dir,'subj_{0:04d}.csv'.format(index))
    # print('Saving to : {}'.format(fn))
    # print(summary)
    # sys.exit()
    # # summary.to_csv(fn)



In [42]:

fn = os.path.join('simul','ground_truth.csv')
params_df = pd.read_csv(fn,index_col=0)

fn = os.path.join('simul','design_set.csv')
design_df = pd.read_csv(fn,index_col=0)

simulated_data = sorted(glob.glob(os.path.join('simul','response','*.csv')))
kappa_hat,gamma_hat = [],[]

for index,fn in enumerate(simulated_data):
    # print(fn)
    df = pd.read_csv(fn,index_col=0)
    choices = df['response']
    kh,gh = estimate_bhm(index,design_df,choices)
    kappa_hat += [kh]
    gamma_hat += [gh]

params_df['kappa_bhm'] = kappa_hat
params_df['gamma_bhm'] = gamma_hat
params_df